# Oanda Demo Trading Notebook

## Packages & Tools

In [8]:
from utils.packages import *
from utils.tools import *
from utils.order_engine import *
from utils.direction_engine import *
from utils.output_log import *

## API Setup

Read from config file

In [35]:
data = {}
config_files = ['access_keys', 'currencies', 'stepped_loss', 'parameters']


for _, val in enumerate(config_files):
    temp_file = f'config/{val}.yaml'
    with open(temp_file) as temp_file:
        data[val] = yaml.load(temp_file)
        
print(data.keys())        

dict_keys(['access_keys', 'currencies', 'stepped_loss', 'parameters'])


In [34]:
access_token = data['access_keys']['oanda_demo_account']['token']
accountID = data['access_keys']['oanda_demo_account']['account_id']
api = API(access_token = access_token)

## Inputs

In [ ]:
tick_speed:
  num_ticks: 300


direction:
  max_pip_levels: 3
  dir_ratio: 2
  min_count: 1
  long_run_multiplier: 2
  short_run_multiplier: 1  
  recalc_multiplier: 5


profit:
  min_profit_pips: 1
  pip_buffer: 0.5 
  stop_loss_pips: 25


order_run:  
  run_multiplier: 25
  

execution:  
  instrument: 'EUR_USD'
  num_bets: 10

In [ ]:
instrument="EUR_USD"

target_num = 3
min_count = 2
min_count_mulitplier = 1

profit_target_num  = 1
price_allowed_buffer = 0.5 
loss_limit_num = 25
max_loss_num = 1
full_loss_ratio = 10

iter_num = 300
dir_min_trans_multiplier_long = 2
dir_min_trans_multiplier_short = 1

run_min_trans_multiplier = 25
dir_recalc_multiplier = 5

num_of_bets = 100

In [ ]:
pip_size = currencies['currs'][instrument]['pip_size']
pip_gap = currencies['currs'][instrument]['pip_gap']

## Currency movement speed compare

In [ ]:
'''
%%time
iter_nums = [100, 200]
for iter_num in iter_nums:
    run_currency_num_check(accountID, currencies, iter_num, api)
    '''

## Code Engine

In [ ]:
%%time
num = get_min_trans_num(instrument, accountID, iter_num, pip_gap, pip_size, api)
dir_min_trans_num_long = round((num * dir_min_trans_multiplier_long),0)
dir_min_trans_num_short = round((num * dir_min_trans_multiplier_short),0)
dir_min_trans_num = dir_min_trans_num_long

run_min_trans_num = round((num * run_min_trans_multiplier),0)

run_flag = True

for i in range(num_of_bets):    
    print(f'-------------------------------Iteration {i+1}-------------------------------')
    if run_flag:
        direction, df_reached_targets, tick_list = run_direction_engine(accountID, instrument, dir_min_trans_num, target_num, min_count, pip_size, pip_gap, min_count_mulitplier)    
        print('df_reached_targets : ',df_reached_targets)
    if direction == 'no_direction':
        print('no_direction : Run again')
        run_flag = True
        dir_min_trans_num = dir_min_trans_num_long
        output_df = get_output_data('', '', '', '', '', '', '',dirc = False) 
        
    else:
        print(f'Direction : {direction}')
        make_order_log, close_order_log, iters, close_reason = run_order_engine(api, accountID, instrument, direction, profit_target_num, loss_limit_num, price_allowed_buffer, run_min_trans_num, pip_size, loss_limits)
        output_df = get_output_data(api, accountID, instrument, make_order_log, close_order_log, iters, close_reason, dirc = True)
        pl_pip = float(output_df['profit_pips'])
        iter_limit = dir_min_trans_num * dir_recalc_multiplier
        print(f'pl:{pl_pip} iter_limit:{iter_limit}')


        if float(pl_pip) >= 0 and iters < iter_limit:
            run_flag = True
            dir_min_trans_num = dir_min_trans_num_short
        else:
            run_flag = True
            dir_min_trans_num = dir_min_trans_num_long

    output_df.to_csv('data/machine_use/output_log_machine_use.csv', mode='a', header=False, index = False)

## Output copy

In [ ]:
df_copy = pd.read_csv('data/machine_use/output_log_machine_use.csv')
df_copy.to_csv('data/output_log.csv', index = False)    